In [1]:
"""
base class

A node has a value and 3 pointers - 
left, right and parent.
"""

class Node:
    def __init__(self,value):
        self._value = value
        self._left = None
        self._right = None
        self._parent = None
    
    @property
    def value(self):
        return self._value
    
    @property
    def left(self):
        return self._left
    
    @left.setter
    def left(self,l):
        self._left = l
    
    @property
    def right(self):
        return self._right
    
    @right.setter
    def right(self,r):
        self._right = r
        
    @property
    def parent(self):
        return self._parent
    
    @parent.setter
    def parent(self,p):
        self._parent = p

In [24]:
"""
binary search tree


Each vertex has only up to 2 children. 

Complexity:
Basic Ops - is proportional to the height of tree, thus O(h)
in worst cases it is O(N)
in average it is O(lgN)

"""

class BST:
    def __init__(self):
        self._root = None
    
    @property
    def root(self):
        return self._root
    
    @root.setter
    def root(self, root):
        self._root = root
    
    # print values of all nodes
    def inorder_walk(self, node):
        if node != None:
            self.inorder_walk(node.left)
            print(node.value)
            self.inorder_walk(node.right)
            
            
    # query a node with specific value
    def search(self, value):
        node = self._root
        while node != None and node.value != value:
            if node.value < value:
                node = node.right
            else:
                node = node.left
        return node
    
    
    # find node of min value
    def minimum(self, node):
        while node and node.left != None:
            node = node.left
        return node
    
    
    # find node of max value
    def maximum(self, node):
        while node and node.right != None:
            node = node.right
        return node
    
    
    # find successore in sorted order
    def successor(self, node):
        if node.right != None:
            return self.minimum(node.right)
        else:
            p = node.parent
            while p != None and p.right == node:
                node = p
                p = p.parent
            return p
    
    
    # find predecessor in sorted order
    def predecessor(self, node):
        if node.left != None:
            return self.maximum(node.right)
        else:
            p = node.parent
            while p != None and p.left == node:
                node = p
                p = p.parent
            return p        

        
    # insert a new node
    def insert(self, node):
        value = node.value
        n = self._root
        parent = None
        while n != None:
            parent = n
            if n.value > value:
                n = n.left
            else:
                n = n.right
        node.parent = parent
        if parent == None:
            self._root = node
        else:
            if parent.value > value:
                parent.left = node
            else:
                parent.right = node
     

    # when v is u's left or right children
    # by transplanting, u's parent's (left/right) child become v, u's parent becomes v's parent
    # but, u's another children is not touched
    def _transplant(self, u, v):
        # u is root
        if u.parent == None:
            self.root = v
        # u is its parent's left children
        elif u.parent.left == u:
            u.parent.left = v
        # u is its parent's right children
        else:
            u.parent.right = v
            
        if v != None:
            v.parent = u.parent
            
    
    # del a node of value 
    def delete(self, value):
        node = self.search(value)
        if node == None:
            return

        # case 1: node has no children, aka leaf node
        # delete right away
        if node.left == None and node.right == None:
            self._transplant(node, None)
            return

        # case 2: node is an internal/root node with one children
        # do one transplant
        if node.left == None:
            self._transplant(node, node.right)
            return
        elif node.right == None:
            self._transplant(node, node.left)  
            return
        
        # case 3: node is an internal/root node with two children
        # replace node with its successor
        suc = self.successor(node)
        # successor is node's children
        if suc.parent == node:
            self._transplant(node, suc)
            suc.left = node.left
            node.left.parent = suc 
        # successor is not node's right children, but in the subtree
        else:
            self._transplant(suc, suc.right)
            suc.right = node.right
            suc.right.parent = suc
            self._transplant(node.suc)
            suc.left = node.left
            node.left.parent = suc

In [26]:
# test bst
bst = BST()
print("insert 0-5")
bst.insert(Node(2))
bst.insert(Node(3))
bst.insert(Node(5))
bst.insert(Node(4))
bst.insert(Node(0))
bst.insert(Node(1))
bst.inorder_walk(bst.root)
print("min {0}".format(bst.minimum(bst.root).value))
print("max {0}".format(bst.maximum(bst.root).value))
four = bst.search(4)
print("predecessor of {0} is {1}".format(four.value, bst.predecessor(four).value))
print("successor of {0} is {1}".format(four.value, bst.successor(four).value))
print("delete 4")
bst.delete(4)
bst.inorder_walk(bst.root)
print("delete 2")
bst.delete(2)
bst.inorder_walk(bst.root)
print("insert 2 back")
bst.insert(Node(2))
bst.inorder_walk(bst.root)

insert 0-5
0
1
2
3
4
5
min 0
max 5
predecessor of 4 is 3
successor of 4 is 5
delete 4
0
1
2
3
5
delete 2
0
1
3
5
insert 2 back
0
1
2
3
5


In [43]:
"""
red-black tree

properties:
1. Every node is either red or black;
2. Root node is black;
3. Every Nil leaf is black;
4. A red node has two black children nodes;
5. Every path from root to leaf has same number of black nodes (black-height).

1 to 1 correspondence to a 2-3 tree
"""

class RBNode(Node):
    
    def __init__(self,value):
        Node.__init__(self, value)
        self._color = None
        self._height = -1
        
    @property
    def color(self):
        return self._color
    
    @property
    def height(self):
        return self._height
    
    @color.setter
    def color(self,color):
        self._color = color
        
    @height.setter
    def height(self,height):
        self._height = height
        
class RBTree(BST):
    
    def __init__(self):
        BST.__init__(self)
        
    def inorder_walk(self, node):
        if node != None:
            self.inorder_walk(node.left)
            print("value={0} color={1} height={2}".format(node.value, node.color, node.height))
            self.inorder_walk(node.right)
#     def insert(self, node):
#         pass
#     def left_rotate(self, node):
#         pass
#     def right_rotate(self, node):
#         pass        

In [44]:
rbt = RBTree()
print("insert 0-5")
rbt.insert(RBNode(2))
rbt.insert(RBNode(3))
rbt.insert(RBNode(5))
rbt.insert(RBNode(4))
rbt.insert(RBNode(0))
rbt.insert(RBNode(1))
rbt.inorder_walk(rbt.root)

insert 0-5
value=0 color=None height=-1
value=1 color=None height=-1
value=2 color=None height=-1
value=3 color=None height=-1
value=4 color=None height=-1
value=5 color=None height=-1
